In [4]:
import pandas as pd
import sqlite3
import numpy as np
import calendar

In [5]:
# create a connection with the given database
con = sqlite3.connect("../laliga.sqlite")

# read the two tables from the database into Pandas DataFrames
df_matches = pd.read_sql_query("SELECT * from Matches", con)
df_predictions = pd.read_sql_query("SELECT * from Predictions", con)

# close the connection of the database
con.close()

In [6]:
df_predictions

,season,division,matchday,date,time,home_team,away_team,score,pred
0,2021-2022,1,3,8/27/21,8:00 PM,RCD Mallorca,Espanyol,1:0,X
1,2021-2022,1,3,8/27/21,10:15 PM,Valencia,Alavés,3:0,X
2,2021-2022,1,3,8/28/21,5:00 PM,Celta de Vigo,Athletic,0:1,X
3,2021-2022,1,3,8/28/21,7:30 PM,Real Sociedad,Levante,1:0,X
4,2021-2022,1,3,8/28/21,7:30 PM,Elche CF,Sevilla FC,1:1,X
5,2021-2022,1,3,8/28/21,10:00 PM,Real Betis,Real Madrid,0:1,X
6,2021-2022,1,3,8/29/21,5:00 PM,Barcelona,Getafe,2:1,X
7,2021-2022,1,3,8/29/21,7:30 PM,Cádiz CF,CA Osasuna,2:3,X
8,2021-2022,1,3,8/29/21,7:30 PM,Rayo Vallecano,Granada CF,4:0,X
9,2021-2022,1,3,8/29/21,10:00 PM,Atlético Madrid,Villarreal,2:2,X


In [3]:
# function change the 2 digit representation of the year in the "date" column to a 4 digit representation
def get_year(season, date):
    # get only the 2 digit year part of the date
    year = date.split("/")[-1]
    
    # split the years of the season into two separate years
    season_years = season.split("-")
    
    # check if the 2 digit year part of the date are equal to the last part of the start year or the end year of the season
    for season_year in season_years:
        # if the 2 digit part is equal to the last part of one of the season years
        if year == season_year[2:4]:
            # return the complete date, now containing a 4 digit year representation
            return f"{date.split("/")[0]}/{date.split("/")[1]}/{season_year}"

In [4]:
# creating two new columns, one for the score of the home team, and one for the score of the away team
df_matches[['home_score', 'away_score']] = df_matches['score'].str.split(':', expand=True).astype(float)

# creating one new column to show which team won (1 for home team, 2 for away team), or if it was a tie (X)
df_matches['winner'] = df_matches.apply(lambda row: 
                                        np.nan if pd.isna(row['home_score']) or pd.isna(row['away_score'])
                                        else 1 if row['home_score'] > row['away_score']
                                        else 2 if row['home_score'] < row['away_score']
                                        else 0, axis=1)

# improve the "date" column using the "get_year" function to change the year from 2 digits to the correct 4 digits
df_matches['date'] = df_matches.apply(lambda row: get_year(row['season'], row['date']), axis=1)


# change the "date" column to datetime
df_matches['date'] = pd.to_datetime(df_matches['date'], format='mixed')


# add a weekday column to store which day of the week the match was played
df_matches['weekday'] = df_matches.apply(lambda row: row['date'].weekday(), axis=1)

df_matches = df_matches.drop(columns=['time'])

df_matches = df_matches.dropna()
df_matches

,season,division,matchday,date,home_team,away_team,score,home_score,away_score,winner,weekday
0,1928-1929,1,1,1929-02-10,Arenas Club,Athletic Madrid,2:3,2.0,3.0,2.0,6
1,1928-1929,1,1,1929-02-10,Espanyol,Real Unión,3:2,3.0,2.0,1.0,6
2,1928-1929,1,1,1929-02-10,Real Madrid,Catalunya,5:0,5.0,0.0,1.0,6
3,1928-1929,1,1,1929-02-10,Donostia,Athletic,1:1,1.0,1.0,0.0,6
4,1928-1929,1,1,1929-02-12,Racing,Barcelona,0:2,0.0,2.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...
48345,2021-2022,2,3,2021-08-28,Ponferradina,Girona,2:1,2.0,1.0,1.0,5
48346,2021-2022,2,3,2021-08-29,SD Amorebieta,UD Almería,2:1,2.0,1.0,1.0,6
48347,2021-2022,2,3,2021-08-29,CD Lugo,Real Valladolid,0:2,0.0,2.0,2.0,6
48348,2021-2022,2,3,2021-08-29,Real Sociedad B,CF Fuenlabrada,0:0,0.0,0.0,0.0,6


In [5]:
df_matches.count()

season        48000
division      48000
matchday      48000
date          48000
home_team     48000
away_team     48000
score         48000
home_score    48000
away_score    48000
winner        48000
weekday       48000
dtype: int64

In [6]:
df_matches.dtypes

season                object
division               int64
matchday               int64
date          datetime64[ns]
home_team             object
away_team             object
score                 object
home_score           float64
away_score           float64
winner               float64
weekday                int64
dtype: object

In [7]:
df_matches.corr(numeric_only=True).style.background_gradient("coolwarm", vmin=-1, vmax=1)

,division,matchday,home_score,away_score,winner,weekday
division,1.000000,0.090943,-0.089428,-0.047485,-0.021553,0.010434
matchday,0.090943,1.000000,-0.055361,-0.007378,0.009374,-0.036487
home_score,-0.089428,-0.055361,1.000000,0.028580,-0.040034,0.048915
away_score,-0.047485,-0.007378,0.028580,1.000000,0.382256,-0.008787
winner,-0.021553,0.009374,-0.040034,0.382256,1.000000,-0.009148
weekday,0.010434,-0.036487,0.048915,-0.008787,-0.009148,1.000000


In [8]:
target = 'winner'
features = ['home_score', 'away_score', 'weekday', 'matchday']
X, y = df_matches[features], df_matches[target]

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (38400, 4)
X_test shape: (9600, 4)
y_train shape: (38400,)
y_test shape: (9600,)


In [10]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

LinearRegression()

In [11]:
linear_model.coef_, linear_model.intercept_

(array([-0.02366033,  0.24906644, -0.0032003 ,  0.00048266]),
 np.float64(0.7625963236661013))

In [12]:
linear_y_pred = linear_model.predict(X_test)

results_df = X_test.copy()
results_df["y_real"] = y_test
results_df["y_pred"] = linear_y_pred.astype(int)
results_df["err"] = results_df["y_real"] - results_df["y_pred"]
results_df["%_err"] = results_df["err"] / results_df["y_real"] * 100
results_df

,home_score,away_score,weekday,matchday,y_real,y_pred,err,%_err
23629,3.0,1.0,5,10,1.0,0,1.0,100.0
43458,2.0,1.0,6,13,1.0,0,1.0,100.0
45365,1.0,1.0,5,19,0.0,0,0.0,NaN
19839,1.0,0.0,2,11,1.0,0,1.0,100.0
10859,1.0,3.0,6,6,2.0,1,1.0,50.0
...,...,...,...,...,...,...,...,...
27942,3.0,0.0,6,23,1.0,0,1.0,100.0
39450,1.0,1.0,6,27,0.0,0,0.0,NaN
35036,2.0,0.0,6,9,1.0,0,1.0,100.0
42373,4.0,1.0,5,41,1.0,0,1.0,100.0


In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

print(f"RMSE: {mean_squared_error(y_test, linear_y_pred)**0.5}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, linear_y_pred)}")
print(f"R^2: {r2_score(y_test, linear_y_pred)}")

RMSE: 0.6317410221199726
MAPE: 1097436808176832.0
R^2: 0.1618302293491336
